<a href="https://colab.research.google.com/github/butchland/fastai_nb_explorations/blob/master/fastai_scratch_with_tpu_mnist_4_experiment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [ ]:
!curl https://course.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   322  100   322    0     0    933      0 --:--:-- --:--:-- --:--:--   933
Updating fastai...
Done.


In [ ]:
VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4264  100  4264    0     0  42640      0 --:--:-- --:--:-- --:--:-- 42640
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200325 ...
Uninstalling torch-1.5.1+cu101:
Done updating TPU runtime: <Response [200]>
  Successfully uninstalled torch-1.5.1+cu101
Uninstalling torchvision-0.6.1+cu101:
  Successfully uninstalled torchvision-0.6.1+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly+20200325-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 83.4 MiB/ 83.4 MiB]                                                
Operation completed over 1 objects/83.4 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200325-cp36-cp36m-linux_x86_64.whl...
- [1 files][114.5 MiB/114.5 MiB]                                                
Operation completed over 1 objec

In [ ]:
!pip freeze | grep torchvision

torchvision==0.6.0a0+3c254fb


In [ ]:
!pip install fastcore --upgrade

In [ ]:
!pip install fastai2 --upgrade

     |████████████████████████████████| 194kB 3.4MB/s 


In [ ]:
pip install fastai --upgrade

Requirement already up-to-date: fastai in /usr/local/lib/python3.6/dist-packages (1.0.61)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/My\ Drive/course-v4/
!pwd

/content/drive/My Drive/course-v4
/content/drive/My Drive/course-v4


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 51kB 1.9MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 1.1MB 11.1MB/s 
     |████████████████████████████████| 92kB 11.0MB/s 
     |████████████████████████████████| 40kB 5.4MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 61kB 7.9MB/s 
     |████████████████████████████████| 2.7MB 34.3MB/s 


In [ ]:
%cd nbs

/content/drive/My Drive/course-v4/nbs


In [ ]:
!pwd

/content/drive/My Drive/course-v4/nbs


### Start of import libraries

In [ ]:
from fastai2.vision.all import *

In [ ]:
from utils import *

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)

In [ ]:
Path.BASE_PATH = path

In [ ]:
path.ls()

(#3) [Path('labels.csv'),Path('valid'),Path('train')]

### Import torch xla libraries

In [ ]:
import torch

In [ ]:
import torch_xla

In [ ]:
import torch_xla.core.xla_model as xm

### define load data tensors in cpu

In [ ]:
def load_tensors(dpath):
    return torch.stack([tensor(Image.open(o)) 
                        for o in dpath.ls().sorted()]
                      ).float()/255.


In [ ]:
def count_images(dpath):
    return len(dpath.ls())

In [ ]:
train_x = torch.cat([load_tensors(path/'train'/'3'), 
                    load_tensors(path/'train'/'7')]).view(-1,28*28)
valid_x = torch.cat([load_tensors(path/'valid'/'3'), 
                    load_tensors(path/'valid'/'7')]).view(-1,28*28)

In [ ]:
(train_x.device, valid_x.device)

(device(type='cpu'), device(type='cpu'))

In [ ]:
(train_x.shape, valid_x.shape)

(torch.Size([12396, 784]), torch.Size([2038, 784]))

In [ ]:
train_y = tensor([1]*count_images(path/'train'/'3') + [0]*count_images(path/'train'/'7')).unsqueeze(1)
valid_y = tensor([1]*count_images(path/'valid'/'3') + [0]*count_images(path/'valid'/'7')).unsqueeze(1)
(train_y.shape, valid_y.shape, train_y.device, valid_y.device)

(torch.Size([12396, 1]),
 torch.Size([2038, 1]),
 device(type='cpu'),
 device(type='cpu'))

In [ ]:
train_dl = DataLoader(list(zip(train_x, train_y)),batch_size=256)
valid_dl = DataLoader(list(zip(valid_x, valid_y)), batch_size=256)

### Get TPU Device

In [ ]:
tpu_dev = xm.xla_device()

In [ ]:
tpu_dev

device(type='xla', index=1)

## Fix Model


In [ ]:
torch.manual_seed(42)
np.random.seed(42)



### Loss function


In [ ]:
# define loss function using sigmoid to return a val between 0.0 and 1
def mnist_loss_sigmoid(qpreds, qtargs):
    qqpreds = qpreds.sigmoid()
    return torch.where(qtargs==1, 1.-qqpreds, qqpreds).mean()



### Forward Pass + Back Propagation


In [ ]:
# forward prop + back prop
def calc_grad(xb,yb,m):
    qpreds = m(xb)
    qloss = mnist_loss_sigmoid(qpreds,yb)
    qloss.backward()


### Basic Optimizer

In [ ]:
class BasicOptimizer:
    def __init__(self, params,lr): self.lr, self.params = lr,list(params)
    def step(self, *args, **kwargs):
        for p in self.params: p.data -= p.grad.data * self.lr
    def zero_grad(self, *args, **kwargs):
        for p in self.params: p.grad = None



### Train Epoch


In [ ]:
def train_epoch(qdl,qmodel,qopt, dev):
    for xb,yb in qdl:
        calc_grad(xb.to(dev),yb.to(dev),qmodel)
        # qopt.step()
        # replace optimizer step with xla device step computation
        xm.optimizer_step(qopt, barrier=True)
        qopt.zero_grad()



### Compute Metrics


In [ ]:
def batch_accuracy(qpreds, qtargets):
    qqpreds = qpreds.sigmoid()
    correct = (qqpreds > 0.5) == qtargets
    return correct.float().mean()


In [ ]:
def validate_epoch(qmodel, qdl, dev):
    accs = [batch_accuracy(qmodel(xb.to(dev)), yb.to(dev)) for xb,yb in qdl]
    return round(torch.stack(accs).mean().item(),4)


In [ ]:
def train_model(qtrain_dl, qvalid_dl, qmodel, qopt, epochs, dev):
    for i in range(epochs):
        train_epoch(qtrain_dl, qmodel, qopt, dev)
        print(validate_epoch(qmodel, qvalid_dl, dev), end=' ')



### Build and Train Model


In [ ]:

model = nn.Linear(28*28,1).to(tpu_dev)


In [ ]:
optim = BasicOptimizer(model.parameters(),0.5)

In [ ]:

# use basic Optimizer
train_model(train_dl, valid_dl, model, optim, 50, tpu_dev)                     


0.5132 0.8247 0.9141 0.9375 0.9517 0.9624 0.9663 0.9678 0.9692 0.9712 0.9717 0.9731 0.9741 0.9761 0.9765 0.9765 0.977 0.977 0.9775 0.9785 0.979 0.979 0.9795 0.9804 0.9804 0.9804 0.9809 0.9809 0.9814 0.9814 0.9824 0.9829 0.9824 0.9824 0.9824 0.9824 0.9824 0.9829 0.9829 0.9824 0.9824 0.9824 0.9829 0.9829 0.9829 0.9829 0.9829 0.9829 0.9829 0.9829 

In [ ]:

train_model(train_dl, valid_dl, model, SGD(model.parameters(),0.1), 50, tpu_dev)                     

0.9824 0.9824 0.9834 0.9834 0.9834 0.9834 0.9834 0.9829 0.9829 0.9829 0.9829 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 

In [ ]:
simple_net = nn.Sequential(
    nn.Linear(28*28,30),
    nn.ReLU(),
    nn.Linear(30,1)
).to(tpu_dev)

In [ ]:
sgd_optim1 = SGD(simple_net.parameters(),0.1)

In [ ]:
train_model(train_dl, valid_dl, simple_net, sgd_optim1, 50, tpu_dev)

0.5132 0.7959 0.9121 0.9404 0.9531 0.9629 0.9663 0.9668 0.9692 0.9707 0.9726 0.9741 0.9746 0.9746 0.9751 0.977 0.9775 0.978 0.978 0.978 0.9795 0.9795 0.9804 0.9804 0.9809 0.9814 0.9819 0.9819 0.9819 0.9819 0.9819 0.9829 0.9829 0.9834 0.9829 0.9829 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9824 0.9829 0.9829 0.9829 0.9829 0.9834 0.9834 0.9834 

In [ ]:
resnet18_model = resnet18(pretrained=True).to(tpu_dev)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth


In [ ]:
sgd_optim18 = SGD(resnet18_model.parameters(), 1e-2)

In [ ]:
train_model(train_dl, valid_dl, resnet18_model, sgd_optim18, 1, tpu_dev)

RuntimeError: ignored